# 27. ReAct: A Abordagem LangChain

No notebook anterior, construímos um agente "na mão". Agora vamos ver como o LangChain abstrai isso para facilitar a construção de sistemas complexos.

**Objetivos:**
1. Usar ferramentas pré-construídas do LangChain.
2. Criar um Agente ReAct padrão (`create_react_agent`).
3. **Engenharia de Prompt Reversa:** Baixar e analisar o prompt padrão do LangChain Hub para entender as boas práticas embutidas nele.

---

# Explicação Detalhada do Assunto

# 27. ReAct: A Abordagem LangChain

Bem-vindo! Neste notebook, vamos mergulhar na implementação do padrão ReAct (Reasoning and Acting) utilizando o LangChain. Se você construiu um agente "na mão" no notebook anterior, prepare-se para ver como o LangChain simplifica drasticamente o processo, permitindo a criação de sistemas complexos com muito mais facilidade.

**Resumo Executivo:**

Este notebook demonstra como utilizar o LangChain para construir um agente ReAct eficaz. Exploraremos a definição de ferramentas, a utilização de prompts pré-definidos do LangChain Hub e a criação e execução do agente. O foco é mostrar como o LangChain abstrai a complexidade da construção de agentes, tornando o processo mais acessível e eficiente.

**Conceitos Chave:**

*   **ReAct (Reasoning and Acting):** Um padrão para agentes de IA que combina raciocínio (reasoning) para gerar planos e ações (acting) para interagir com o ambiente.
*   **LangChain Hub:** Um repositório de prompts, chains e agentes pré-construídos e testados pela comunidade, que podem ser facilmente reutilizados.
*   **Ferramentas (Tools):** Funções específicas que o agente pode utilizar para interagir com o mundo externo, como buscar informações na internet ou calcular o tamanho de uma palavra.
*   **`@tool` Decorator:** Uma funcionalidade do LangChain que simplifica a criação de ferramentas, injetando automaticamente a documentação da função no prompt do agente.
*   **AgentExecutor:** O componente do LangChain responsável por executar o loop de raciocínio e ação do agente.

**Objetivos de Aprendizado:**

Ao concluir este notebook, você será capaz de:

*   Definir e integrar ferramentas personalizadas em um agente LangChain utilizando o decorador `@tool`.
*   Utilizar prompts pré-definidos do LangChain Hub para acelerar o desenvolvimento de agentes.
*   Construir um agente ReAct completo utilizando as funcionalidades do LangChain.
*   Compreender o fluxo de execução de um agente LangChain, incluindo o raciocínio, a seleção de ferramentas e a interação com o ambiente.
*   Interpretar o output detalhado (verbose) do agente para entender seu processo de tomada de decisão.

**Importância no Ecossistema LangChain:**

O padrão ReAct é fundamental para a construção de agentes inteligentes e capazes de resolver problemas complexos. O LangChain fornece as ferramentas e abstrações necessárias para implementar o ReAct de forma eficiente, permitindo que você se concentre na lógica de negócio em vez de se preocupar com os detalhes de implementação. Dominar a construção de agentes ReAct com LangChain é um passo essencial para aproveitar ao máximo o poder da IA generativa.

## 1. Definindo Ferramentas no LangChain

O LangChain facilita a criação de ferramentas robustas usando o decorador `@tool`. O docstring da função é **CRUCIAL**, pois ele é injetado no prompt para o LLM saber como usar a ferramenta!

## 2. O Prompt do LangChain Hub

Ao invés de escrevermos o prompt template gigante (como no notebook anterior), vamos puxar o padrão da comunidade testado em batalha.

O prompt `hwchase17/react` é o padrão para o ReAct.

### Análise do Prompt

Note que ele possui variáveis como `{tools}`, `{tool_names}` e `{agent_scratchpad}`.

*   **{tools}**: Descrições das ferramentas (automático).
*   **{tool_names}**: Lista de nomes das ferramentas (automático).
*   **{agent\_scratchpad}**: Histórico das interações do agente (automático).

## 3. Criando o Agente (Agent Construction)

Vamos usar `create_react_agent`. O "Executor" é quem roda o loop (o equivalente ao nosso `run_agent_step` manual).

## 4. Rodando o Agente

Vamos ver o `verbose=True` em ação, mostrando o pensamento.

---


In [1]:
### INJECTION START ###
import os
from dotenv import load_dotenv
import sys
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
### INJECTION END ###

# !pip install -q langchain langchain-openai langchainhub google-search-results numexpr # Script-patched

In [2]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
    
# Opcional: SerpAPI para busca real no Google
# if "SERPAPI_API_KEY" not in os.environ:
    pass # Script-patched: ensure non-empty block
#     os.environ["SERPAPI_API_KEY"] = os.getenv("GOOGLE_API_KEY"): ")

## 1. Definindo Ferramentas no LangChain

O LangChain facilita a criação de ferramentas robustas usando o decorador `@tool`. O docstring da função é **CRUCIAL**, pois ele é injetado no prompt para o LLM entender quando usar a ferramenta.

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Retorna o tamanho (número de caracteres) de uma palavra."""
    return len(word)

@tool
def get_weather(city: str) -> str:
    """Retorna a previsão do tempo para uma cidade específica. Use para perguntas sobre clima."""
    # Mock para exemplo
    return f"O tempo em {city} está ensolarado, 25 graus."

tools = [get_word_length, get_weather]

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 2. O Prompt do LangChain Hub

Ao invés de escrevermos o prompt template gigante (como no notebook anterior), vamos puxar o padrão da comunidade testado em batalha.

O prompt `hwchase17/react` é o padrão *gold standard*.

In [4]:
from langchain import hub

# Baixando o prompt
prompt = hub.pull("hwchase17/react")

# Vamos imprimir para estudar sua estrutura
print("--- Prompt Template Padrão ---")
print(prompt.template)

--- Prompt Template Padrão ---
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Análise do Prompt

Note que ele possui variáveis como `{tools}`, `{tool_names}` e `{agent_scratchpad}`.

- **{tools}**: Descrições das ferramentas (automático).
- **{tool_names}**: Lista de nomes (automático).
- **{agent_scratchpad}**: Onde o histórico de Pensamento/Ação/Observação é injetado para manter o contexto.

## 3. Criando o Agente (Agent Construction)

Vamos usar `create_react_agent`. O "Executor" é quem roda o loop (o equivalente ao nosso `run_agent_step` manual).

In [5]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Criação do Agente (a "mente" que decide o que fazer baseada no prompt)
agent = create_react_agent(llm, tools, prompt)

# Criação do Executor (o "corpo" que executa as ações e loops)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# handle_parsing_errors=True é uma técnica de engenharia de prompt automática!
# Se o LLM falhar no formato, o LangChain injeta uma instrução de erro no prompt pedindo para corrigir.

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/oauth2/__init__.py:40: FutureWarning: You are using a Python version 3.9 past its end of life. Google will upda

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/langchain_google_genai/chat_models.py:47: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  from google.generativeai.caching import CachedContent  # type: ignore[import]


## 4. Rodando o Agente

Vamos ver o `verbose=True` em ação, mostrando o pensamento.

In [6]:
query = "Quantas letras tem a palavra 'paralelepípedo' e como está o tempo no Rio de Janeiro?"

response = agent_executor.invoke({"input": query})

print(f"\nResposta: {response['output']}")



> Entering new AgentExecutor chain...


Thought: The question has two parts. First, I need to find the length of the word 'paralelepípedo'. Second, I need to get the weather in Rio de Janeiro.
Action: get_word_length
Action Input: paralelepípedo14

I have the length of the word. Now I need to get the weather in Rio de Janeiro.
Action: get_weather
Action Input: Rio de JaneiroO tempo em Rio de Janeiro está ensolarado, 25 graus.

I have both answers.
Final Answer: A palavra 'paralelepípedo' tem 14 letras e o tempo no Rio de Janeiro está ensolarado, 25 graus.

> Finished chain.

Resposta: A palavra 'paralelepípedo' tem 14 letras e o tempo no Rio de Janeiro está ensolarado, 25 graus.


## 5. Exercício Prático: Modificando o Prompt Base

E se quisermos que o agente sempre responda como um pirata?
Podemos injetar instruções no topo do prompt.

In [7]:
# Modificando o template manualmente
prompt.template = "VOCÊ É UM PIRATA REBULIÇO! SEMPRE RESPONDA COMO TAL.\n\n" + prompt.template

agent_pirata = create_react_agent(llm, tools, prompt)
executor_pirata = AgentExecutor(agent=agent_pirata, tools=tools, verbose=True)

executor_pirata.invoke({"input": "Como está o tempo em São Paulo?"})



> Entering new AgentExecutor chain...


Arrr, um pirata precisa saber o tempo, hein? Preciso usar meu telescópio mágico para descobrir!

Thought: Preciso usar a ferramenta para saber o clima em São Paulo.
Action: get_weather
Action Input: São PauloO tempo em São Paulo está ensolarado, 25 graus.

Arrr, agora eu sei!

Final Answer: Arrr, o tempo em São Paulo está ensolarado, 25 graus!

> Finished chain.


{'input': 'Como está o tempo em São Paulo?',
 'output': 'Arrr, o tempo em São Paulo está ensolarado, 25 graus!'}